In [1]:
import fundamentalanalysis as fa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
import statistics as st

%matplotlib inline 

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
api_key = '00ef9804fcde0edd93b1b4821ee2f06a'
# api_key = 'c81352430e2fe3c941faf0814227562b'

In [4]:
''' Колонки '''
ticker = 'AAPL'
# ticker = 'AAPL'

# tickers_list = ['AXP', 'INTC']
# tickers_list = ['ADBE', 'AAPL', 'F']
# tickers_list = ['WBA', 'IRBT', 'WRK', 'IRBT', 'CSCO', 'KR']
# tickers_list = ['UAA', 'HBAN', 'HPE', 'IVZ', 'KEY', 'RF', 'BEN', 'WMB', 'DAL', 'GM', 'WRK', 'WFC', 'EBAY', 'VTR', 'AIG', 'MU', 'MET', 'NTAP', 'SYY', 'BXP', 'MNST', 'XOM', 'PYPL', 'DIS', 'TROW', 'HLT', 'QCOM', 'CVX', 'AAPL', 'HON', 'UPS', 'LOW', 'ACN', 'SNPS', 'LRCX', 'BKNG']

tickers_list = ['F', 'WU', 'KMI', 'CNP', 'HPQ', 'GLW', 'KHC', 'LUV', 'CMCSA', 'NRG', 'BK', 'CSCO', 'USB', 'TJX', 'CAH', 'HIG', 'SCHW', 'HAS', 'WELL']
# tickers_list = ['F', 'WU', 'KMI', 'CNP', 'HPQ', 'GLW', 'KHC', 'LUV', 'CMCSA', 'NRG', 'BK', 'CSCO', 'USB', 'TJX', 'CAH', 'HIG', 'SCHW', 'HAS', 'WELL', 'DXCM', 'ROST', 'GRMN', 'NKE', 'YUM', 'GOOGL', 'JPM', 'GOOG', 'GPN', 'MMM', 'AXP', 'TGT', 'TXN', 'CRM', 'NDAQ', 'CAT', 'ILMN', 'V', 'GD', 'NFLX', 'MSFT', 'HD', 'MA', 'SPGI', 'IDXX', 'ADBE', 'AVGO',
# 'UAA', 'HBAN', 'HPE', 'IVZ', 'KEY', 'RF', 'BEN', 'WMB', 'DAL', 'GM', 'WRK', 'WFC', 'EBAY', 'VTR', 'AIG', 'MU', 'MET', 'NTAP', 'SYY', 'BXP', 'MNST', 'XOM', 'PYPL', 'DIS', 'TROW', 'HLT', 'QCOM', 'CVX', 'AAPL', 'HON', 'UPS']
# tickers_list = ['LRCX', 'BKNG','KEY', 'BAC', 'LVS', 'UAL', 'HOG', 'CFG', 'ALK', 'MAS', 'BBY', 'EMR', 'PSX', 'MS', 'LYB', 'RTX', 'RL', 'COP', 'AMAT', 'QRVO', 'SPG', 'DRI', 'BA', 'META', 'NVDA', 'FDX', 'ADSK', 'ABMD', 'GS',
# 'GPS', 'XRX', 'BKR', 'PARA', 'FCX', 'ALK', 'WDC', 'C', 'MOS', 'OKE', 'DHI', 'BBY', 'MPC', 'AMD', 'PRU', 'EXPE', 'EOG', 'VLO', 'MAR', 'HCA', 'ALB']


required_year = '2022'


''' время '''
stime='1990-01-01'
ftime='2023-02-01'
period = '1mo'  # '1d' '1wk'

In [5]:
def one_stock_analyse(ticker, api_key, required_year):

    def get_data_from_ticker(tick, interval='1mo'):
        ticker = yf.Ticker(tick)

        df = ticker.history(start='1990-01-01', end=None, interval=interval)
        x = pd.DataFrame(df)
        x.rename(columns={"Close": tick}, inplace=True)
        z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
        return z

    
    df = get_data_from_ticker(ticker)



    def change_index(df):
        df_index = df.index.to_list()
        for i in range(len(df_index)):
            t = df_index[i].date()
            df_index[i] = t.strftime('%Y-%m-%d')
        df['Date_index'] = df_index
        df = df.set_index('Date_index')
        return df

    df = change_index(df)


    def get_key_metricks(ticker, api_key, required_year):
        key_metrics = fa.key_metrics(ticker, api_key, period='annual')
        df_key_metr = key_metrics.T
        df_key_metr = df_key_metr.loc[required_year:'2000'].copy(deep=True)
        return df_key_metr



    df_key_metr = get_key_metricks(ticker, api_key, required_year)


    if df_key_metr.index[0] == required_year and f'{int(required_year)+1}-01-01' in df.index.to_list():
        last_year = True
    else:
        last_year = False



    def get_index(df):
        index_years = df.index.to_list()
        return index_years

    index_years = get_index(df_key_metr)



    def get_price_from_df(df, list_dates):
        new_list = []
        for i in range(len(list_dates)):
            x = int(list_dates[i])
            x += 1
            x = str(x)
            try:
                r = df.loc[f'{x}-01-01']
                r = float(r)
                new_list.append(r)
            except:
                new_list.append(0)

        return new_list


    stock_year_prices = get_price_from_df(df, index_years)


    df_key_metr[ticker] = list(stock_year_prices)
    df_key_metr.drop('period', axis=1, inplace=True)


    def change_percent_all_columns(df):
        x = df.columns.to_list()
        for i in range(len(x)):
            v = df[x[i]].to_list()
            l = []
            for y in range(len(v)):
                try:
                    if y != len(v):
                        vv = (v[y] / (v[y+1] / 100)) - 100
                        l.append(vv)
                    else:
                        vv = 0
                        l.append(vv)
                except:
                    l.append(0)
            df[f'change % {x[i]}'] = l
        return df

    df_with_changes = change_percent_all_columns(df_key_metr)
    df_with_changes = df_with_changes.reindex(index=df_with_changes.index[::-1])


    df_some_columns = df_with_changes[[f'change % {ticker}', 'change % debtToAssets']]


    mean_result = st.mean(df_some_columns[f'change % {ticker}'].to_list()[1:])   
    
    
    df_with_changes.index.name='Date'
    df_with_changes = df_with_changes.reset_index()


    
    l = []

    for i in range(len(df_some_columns['change % debtToAssets'])):
        if i <= 1:
            pass
        elif i == len(df_some_columns['change % debtToAssets']) - 1:
            pass
        elif df_with_changes[f'change % {ticker}'][i] < df_with_changes[f'change % {ticker}'][i-1] and df_with_changes['change % debtToAssets'][i] > df_with_changes['change % debtToAssets'][i-1] and df_with_changes['change % debtToAssets'][i] > df_with_changes[f'change % {ticker}'][i]:
            l.append([ticker, df_with_changes['Date'][i], df_with_changes[f'change % {ticker}'][i+1]])
        
        else:
            pass  
    
    df_means_results = pd.DataFrame(l, columns = ['Ticker', 'Date', 'Result'])
    return df_means_results

In [6]:
# one_stock_analyse(ticker, api_key, required_year)

In [7]:
def list_stock_analyse(tickers_list, api_key, required_year):
    l = []
    for i in range(len(tickers_list)):
        try:
            if i == 0:
                df = one_stock_analyse(tickers_list[i], api_key, required_year)
            else:
                df = pd.concat([df, one_stock_analyse(tickers_list[i], api_key, required_year)], ignore_index=True)
        except:
            pass
    return df


df_means_results = list_stock_analyse(tickers_list, api_key, required_year)

In [8]:
df_means_results

,Ticker,Date,Result
0,F,2005,-2.071733
1,F,2008,479.679238
2,F,2018,6.881481
3,WU,2008,36.149547
4,WU,2012,11.552648
5,WU,2017,-8.762158
6,WU,2020,-11.353143
7,KMI,2013,26.512997
8,KMI,2015,39.641870
9,KMI,2017,5.045523


In [9]:
def change_percent_all_columns(df):
    x = df.columns.to_list()
    for i in range(len(x)):
        v = df[x[i]].to_list()
        l = []
        for y in range(len(v)):
            try:
                if y != len(v):
                    if v[y] < 0 and v[y+1] > 0:
                        l.append('Minus')
                    elif v[y] > 0 and v[y+1] < 0:
                        l.append('Plus')
                    else:
                        vv = (v[y] / (v[y+1] / 100)) - 100
                        l.append(vv) 
                else:
                    vv = 0
                    l.append(vv)
                    # print(vv)
            except:
                l.append(0)
        df[f'change % {x[i]}'] = l
    return df

add key metrics

In [10]:
'''
НАШЕЛ
revenuePerShare < -10 = повышает шансы к лонгу
netIncomePerShare == Minus = повышает шансы к Лонгу
netIncomePerShare == Plus = повышает шансы к Шорту
bookValuePerShare > 100 = повышает шансы к Лонгу
capexPerShare > 100 = повышает шансы к Лонгу
daysPayablesOutstanding > 50 = повышает шансы к Лонгу




НE НАШЕЛ корреляции с годовой стратегией дивиргениции AssetsDebt и Рынка

'receivablesTurnover'
'inventoryTurnover'
'operatingCashFlowPerShare'
'freeCashFlowPerShare'
'shareholdersEquityPerShare'
tangibleBookValuePerShare

'''



"\nНАШЕЛ\nrevenuePerShare < -10 = повышает шансы к лонгу\nnetIncomePerShare == Minus = повышает шансы к Лонгу\nnetIncomePerShare == Plus = повышает шансы к Шорту\nbookValuePerShare > 100 = повышает шансы к Лонгу\ncapexPerShare > 100 = повышает шансы к Лонгу\ndaysPayablesOutstanding > 50 = повышает шансы к Лонгу\n\n\n\n\nНE НАШЕЛ корреляции с годовой стратегией дивиргениции AssetsDebt и Рынка\n\n'receivablesTurnover'\n'inventoryTurnover'\n'operatingCashFlowPerShare'\n'freeCashFlowPerShare'\n'shareholdersEquityPerShare'\ntangibleBookValuePerShare\n\n"

In [11]:
# def add_key_metrics(df):
#     v = 'revenuePerShare'
#     v1 = 'netIncomePerShare'
#     v2 = 'inventoryTurnover'
#     v3 = 'capexPerShare'
#     v4 = 'cashPerShare'
#     v5 = 'bookValuePerShare'
#     v6 = 'daysPayablesOutstanding'
#     v7 = 'receivablesTurnover'
#     l = []
#     l1 = []
#     l2 = []
#     l3 = []
#     l4 = []
#     l5 = []
#     l6 = []
#     l7 = []
#     for i in range(len(df['Ticker'])):
#         if i == 0:
#             key_metrics = fa.key_metrics(df['Ticker'][i], api_key, period='annual').T
#             # print(key_metrics)
#             key_metrics_change = change_percent_all_columns(key_metrics)
#             date = df['Date'][i]
#             l.append(key_metrics_change.loc[date, f'change % {v}'])
#             l1.append(key_metrics_change.loc[date,f'change % {v1}'])
#             l2.append(key_metrics_change.loc[date,f'change % {v2}'])
#             l3.append(key_metrics_change.loc[date,f'change % {v3}'])
#             l4.append(key_metrics_change.loc[date,f'change % {v4}'])
#             l5.append(key_metrics_change.loc[date,f'change % {v5}'])
#             l6.append(key_metrics_change.loc[date,f'change % {v6}'])
#             l7.append(key_metrics_change.loc[date,f'change % {v7}'])
#         elif df['Ticker'][i] == df['Ticker'][i-1]:
#             date = df['Date'][i]
#             l.append(key_metrics_change.loc[date, f'change % {v}'])
#             l1.append(key_metrics_change.loc[date,f'change % {v1}'])
#             l2.append(key_metrics_change.loc[date,f'change % {v2}'])
#             l3.append(key_metrics_change.loc[date,f'change % {v3}'])
#             l4.append(key_metrics_change.loc[date,f'change % {v4}'])
#             l5.append(key_metrics_change.loc[date,f'change % {v5}'])
#             l6.append(key_metrics_change.loc[date,f'change % {v6}'])
#             l7.append(key_metrics_change.loc[date,f'change % {v7}'])
#         else:
#             key_metrics = fa.key_metrics(df['Ticker'][i], api_key, period='annual').T
#             key_metrics_change = change_percent_all_columns(key_metrics)
#             date = df['Date'][i]
#             l.append(key_metrics_change.loc[date, f'change % {v}'])
#             l1.append(key_metrics_change.loc[date,f'change % {v1}'])
#             l2.append(key_metrics_change.loc[date,f'change % {v2}'])
#             l3.append(key_metrics_change.loc[date,f'change % {v3}'])
#             l4.append(key_metrics_change.loc[date,f'change % {v4}'])
#             l5.append(key_metrics_change.loc[date,f'change % {v5}'])
#             l6.append(key_metrics_change.loc[date,f'change % {v6}'])
#             l7.append(key_metrics_change.loc[date,f'change % {v7}'])
#     df[f'change % {v}'] = l
#     df[f'change % {v1}'] = l1
#     df[f'change % {v2}'] = l2
#     df[f'change % {v3}'] = l3
#     df[f'change % {v4}'] = l4
#     df[f'change % {v5}'] = l5
#     df[f'change % {v6}'] = l6
#     df[f'change % {v7}'] = l7
    
#     return df

# df_with_parameters = add_key_metrics(df_means_results)

add income_statement

In [12]:
def add_income_statement(df):
    v = 'revenue'
    v1 = 'costOfRevenue'
    v2 = 'grossProfit'
    v3 = 'researchAndDevelopmentExpenses'
    v4 = 'costAndExpenses'
    v5 = 'ebitda'
    v6 = 'operatingIncome'
    v7 = 'eps'
    l = []
    l1 = []
    l2 = []
    l3 = []
    l4 = []
    l5 = []
    l6 = []
    l7 = []
    for i in range(len(df['Ticker'])):
        if i == 0:
            key_metrics = fa.income_statement(df['Ticker'][i], api_key, period='annual').T
            # print(key_metrics)
            key_metrics_change = change_percent_all_columns(key_metrics)
            date = df['Date'][i]
            l.append(key_metrics_change.loc[date, f'change % {v}'])
            l1.append(key_metrics_change.loc[date,f'change % {v1}'])
            l2.append(key_metrics_change.loc[date,f'change % {v2}'])
            l3.append(key_metrics_change.loc[date,f'change % {v3}'])
            l4.append(key_metrics_change.loc[date,f'change % {v4}'])
            l5.append(key_metrics_change.loc[date,f'change % {v5}'])
            l6.append(key_metrics_change.loc[date,f'change % {v6}'])
            l7.append(key_metrics_change.loc[date,f'change % {v7}'])
        elif df['Ticker'][i] == df['Ticker'][i-1]:
            date = df['Date'][i]
            l.append(key_metrics_change.loc[date, f'change % {v}'])
            l1.append(key_metrics_change.loc[date,f'change % {v1}'])
            l2.append(key_metrics_change.loc[date,f'change % {v2}'])
            l3.append(key_metrics_change.loc[date,f'change % {v3}'])
            l4.append(key_metrics_change.loc[date,f'change % {v4}'])
            l5.append(key_metrics_change.loc[date,f'change % {v5}'])
            l6.append(key_metrics_change.loc[date,f'change % {v6}'])
            l7.append(key_metrics_change.loc[date,f'change % {v7}'])
        else:
            key_metrics = fa.income_statement(df['Ticker'][i], api_key, period='annual').T
            key_metrics_change = change_percent_all_columns(key_metrics)
            date = df['Date'][i]
            l.append(key_metrics_change.loc[date, f'change % {v}'])
            l1.append(key_metrics_change.loc[date,f'change % {v1}'])
            l2.append(key_metrics_change.loc[date,f'change % {v2}'])
            l3.append(key_metrics_change.loc[date,f'change % {v3}'])
            l4.append(key_metrics_change.loc[date,f'change % {v4}'])
            l5.append(key_metrics_change.loc[date,f'change % {v5}'])
            l6.append(key_metrics_change.loc[date,f'change % {v6}'])
            l7.append(key_metrics_change.loc[date,f'change % {v7}'])
    df[f'change % {v}'] = l
    df[f'change % {v1}'] = l1
    df[f'change % {v2}'] = l2
    df[f'change % {v3}'] = l3
    df[f'change % {v4}'] = l4
    df[f'change % {v5}'] = l5
    df[f'change % {v6}'] = l6
    df[f'change % {v7}'] = l7
    
    return df

df_with_parameters = add_income_statement(df_means_results)

In [13]:
df_with_parameters.sort_values(by=['Result'], ascending=False)

,Ticker,Date,Result,change % revenue,change % costOfRevenue,change % grossProfit,change % researchAndDevelopmentExpenses,change % costAndExpenses,change % ebitda,change % operatingIncome,change % eps
1,F,2008,479.679238,-15.179612,-10.859335,-35.804205,0.000000,-9.296148,26.671491,Minus,368.115942
22,GLW,2004,122.577715,24.724919,8.835341,66.666667,3.197674,-3.192075,Minus,121.832061,766.666667
49,TJX,2008,98.837722,7.138839,6.574576,8.917917,0.000000,7.829231,2.354932,-1.689066,6.748466
56,HIG,2008,84.975161,-64.427381,0.000000,0,0.000000,-36.972297,Minus,0,Minus
20,HPQ,2012,79.900494,-5.413179,-5.274329,-5.868892,4.456054,-4.132716,Minus,Minus,Minus
11,CNP,2002,68.082209,-82.861321,-90.036736,-36.344233,0.000000,-85.095952,Minus,-33.164548,Minus
60,HAS,2002,65.794624,-1.404210,-10.161236,5.157344,-54.146017,-1.883970,-86.856762,3.767094,Minus
58,SCHW,2015,63.088329,5.203171,0.000000,0.0,0.000000,0.000000,9.06457,0.0,8.333333
64,WELL,2002,62.389014,20.720841,0.000000,0.0,0.000000,27.431840,19.094585,17.789239,-2.597403
15,HPQ,2004,61.243622,9.367515,12.037432,1.879817,-3.997809,9.248162,21.717452,45.959945,39.759036
